In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils.etoile_ventes_avis import graphe_etoile_ventes_avis
import seaborn as sns
DATA_PATH = "../donnees/ecommerce/"

In [ ]:
dot_ventes = graphe_etoile_ventes_avis()
dot_ventes

In [ ]:
orders    = pd.read_parquet(DATA_PATH + "orders.parquet")
payments  = pd.read_parquet(DATA_PATH + "payments.parquet")
reviews   = pd.read_parquet(DATA_PATH + "reviews.parquet")
customers = pd.read_parquet(DATA_PATH + "customers.parquet")
geo       = pd.read_parquet(DATA_PATH + "geolocation.parquet")

In [ ]:
# == Diagnostic des scores dans 'reviews' ==

nb_total = len(reviews)
nb_anormaux = (reviews["score"] > 5).sum()

print(f"Lignes totales : {nb_total}")
print(f"Lignes avec score > 5 : {nb_anormaux}")

print("Valeurs distinctes détectées :")
print(sorted(reviews["score"].dropna().unique()))
print()

In [ ]:
# === CLEAN DES SCORES D'AVIS ===
print("== Nettoyage des scores dans 'reviews' ==")

# Scores autorisés : 1 à 5 ou NaN
mask_scores_valides = reviews["score"].isna() | (
    (reviews["score"] >= 1) & (reviews["score"] <= 5)
)

nb_total = len(reviews)
nb_valid = mask_scores_valides.sum()
nb_drop  = nb_total - nb_valid

print(f"Lignes totales : {nb_total}")
print(f"Lignes conservées (score 1 à 5 ou NaN) : {nb_valid}")
print(f"Lignes supprimées (score > 5) : {nb_drop}")

# On garde uniquement les lignes valides
reviews = reviews[mask_scores_valides].copy()

print("Valeurs distinctes après nettoyage :")
print(sorted(reviews["score"].dropna().unique()))
print()

In [ ]:
print("Shapes brutes apres cleans :")
print("orders   :", orders.shape)
print("payments :", payments.shape)
print("reviews  :", reviews.shape)
print("customers:", customers.shape)
print("geo      :", geo.shape)

In [ ]:
# 1) Préparation des paiements (pivot par type de paiement)
pay = payments.rename(columns=lambda c: c.replace("payment_", ""))

pay_pivot = (
    pay
    .pivot_table(
        index="order_id",
        columns="type",
        values=["installments", "value"],
        aggfunc={"installments": "max", "value": "sum"}
    )
)

pay_pivot.columns = [f"{k}_{t}" for (k, t) in pay_pivot.columns]
pay_pivot = pay_pivot.rename(columns=lambda c: c.replace("installments_", "int_"))

value_cols = [c for c in pay_pivot.columns if c.startswith("value_")]
pay_pivot["value"] = pay_pivot[value_cols].sum(axis=1)

pay_pivot = pay_pivot.reset_index()


# 2) Préparation des avis (colonnes score, délais, longueur commentaire)
rev = reviews.copy()

base_cols = ["order_id"]
for c in ["score", "creation", "answer", "comment"]:
    if c in rev.columns:
        base_cols.append(c)

detail_cols = [
    c for c in rev.columns
    if (
        c.startswith("score_")
        or c.startswith("creation_")
        or c.startswith("answer_")
        or (c.startswith("comment_") and c != "comment")
    )
]

cols_reviews = list(dict.fromkeys(base_cols + detail_cols))

reviews_dim = rev[cols_reviews].copy()
if "comment" in reviews_dim.columns:
    reviews_dim = reviews_dim.rename(columns={"comment": "comment_length"})


# 3) Préparation des infos client (clé client + zip_code) + Correction des durées dans orders (en mémoire)
cust_fact = customers[["customer_id", "zip_code"]].copy()

orders = orders.copy()

delta_approuvee = orders["approved_at"] - orders["purchase_timestamp"]
delta_envoyee   = orders["delivered_carrier"] - orders["purchase_timestamp"]
delta_livree    = orders["delivered_customer"] - orders["purchase_timestamp"]
delta_estimee   = orders["estimated_delivery"] - orders["purchase_timestamp"]

orders["approuvee"] = delta_approuvee.dt.total_seconds() / 3600
orders["envoyee"]   = delta_envoyee.dt.total_seconds() / 3600
orders["livree"]    = delta_livree.dt.total_seconds() / 3600
orders["estimee"]   = delta_estimee.dt.total_seconds() / 3600


# 4) Construction de la table de faits F_VENTES_AVIS (jointures)
fact_orders_cols = [
    "order_id",
    "customer_id",
    "purchase_timestamp",
    "order_status",
    "approuvee",
    "envoyee",
    "livree",
    "estimee",
]
fact_orders_cols = [c for c in fact_orders_cols if c in orders.columns]

fact_orders = orders[fact_orders_cols].copy()

F_VENTES_AVIS = (
    fact_orders
    .merge(cust_fact,   on="customer_id", how="left")
    .merge(pay_pivot,   on="order_id",   how="left")
    .merge(reviews_dim, on="order_id",   how="left")
)

cols_schema = [
    "order_id",
    "customer_id",
    "zip_code",
    "purchase_timestamp",
    "order_status",
    "value",
    "approuvee",
    "envoyee",
    "livree",
    "estimee",
    "score",
    "creation",
    "answer",
    "comment_length",
]

extra_cols = [
    c for c in F_VENTES_AVIS.columns
    if c.startswith("value_")
    or c.startswith("int_")
    or c.startswith("score_")
    or c.startswith("creation_")
    or c.startswith("answer_")
    or (c.startswith("comment_") and c != "comment_length")
]

final_cols = [c for c in cols_schema + extra_cols if c in F_VENTES_AVIS.columns]
F_VENTES_AVIS = F_VENTES_AVIS[final_cols]


# 5) Contrôle rapide du résultat
print("F_VENTES_AVIS shape :", F_VENTES_AVIS.shape)
print("Colonnes (", len(F_VENTES_AVIS.columns), "):")
print(list(F_VENTES_AVIS.columns))

display(F_VENTES_AVIS.head(5))


# 6) Sauvegarde en Parquet pour les autres analyses
OUTPUT_PATH = "f_ventes_avis.parquet"

F_VENTES_AVIS.to_parquet(OUTPUT_PATH, index=False)
print(f"F_VENTES_AVIS sauvegardé dans : {OUTPUT_PATH}")


In [ ]:
# Analyse 1 : Performance commerciale & saisonnalité vs satisfaction
# Dans cette première analyse, On veux comprendre la dynamique globale du business :
# comment évoluent les ventes dans le temps (année, trimestre, mois) et comment cela
# se connecte à la satisfaction client (score moyen des avis). C’est la vue d’ensemble
# indispensable avant d’entrer dans le détail opérationnel : je pars du CA total et du
# volume de commandes, puis je descends dans les hiérarchies de temps et de géographie
# pour identifier les saisons fortes, les périodes faibles et les zones clés.

In [ ]:
# Hypothèse 1
# On suppose qu’il existe un lien entre l’évolution des ventes et celle des avis :
# lorsque les volumes augmentent fortement (périodes de forte demande), la satisfaction
# pourrait se dégrader, notamment si les délais de livraison s’allongent ou si certaines
# régions sont plus difficiles à desservir. On s’attend à voir une relation entre
# saisonnalité, géographie et score moyen.

In [ ]:
# Préparation
fva = F_VENTES_AVIS.copy()
fva["year"] = fva["purchase_timestamp"].dt.year
fva["month"] = fva["purchase_timestamp"].dt.month
fva["year_month"] = fva["purchase_timestamp"].dt.to_period("M").dt.to_timestamp()


In [ ]:
fig, ax1 = plt.subplots(figsize=(14, 6))

# CA : barres en bleu (axe de gauche)
ax1.bar(monthly["year_month"], monthly["revenue"], color="C0", label="Chiffre d'affaires")
ax1.set_xlabel("Mois")
ax1.set_ylabel("Chiffre d'affaires", color="C0")
ax1.tick_params(axis="y", labelcolor="C0")

# Volume : courbe en orange (axe de droite)
ax2 = ax1.twinx()
ax2.plot(monthly["year_month"], monthly["orders_count"],
         marker="o", color="C1", label="Nombre de commandes")
ax2.set_ylabel("Nombre de commandes", color="C1")
ax2.tick_params(axis="y", labelcolor="C1")

plt.title("Évolution mensuelle du CA et du volume de commandes")

# Légende combinée
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc="upper left")

plt.tight_layout()
plt.show()


In [ ]:
# Le chiffre d’affaires suit de très près la dynamique du nombre de commandes,
# ce qui montre que la croissance vient surtout de l’augmentation des volumes
# plutôt que d’un panier moyen en forte variation.
# On observe une montée rapide jusqu’à fin 2017, avec un pic net en novembre,
# très probablement lié au Black Friday et aux achats de fin d’année anticipés.
# L’activité reste ensuite élevée en 2018, tandis que la satisfaction demeure
# globalement stable. La baisse après août 2018 vient simplement de la fin du dataset.


In [ ]:
# 2) Évolution mensuelle du score moyen des avis

monthly_score = (
    fva
    .groupby("year_month", as_index=False)
    .agg(
        avg_score=("score", "mean"),
        orders_count=("order_id", "nunique")
    )
)

# On se limite aux mois avec un volume suffisant
ms_robust = monthly_score[monthly_score["orders_count"] >= 100].copy()

print("== 2) Score moyen par mois (>= 100 commandes) ==")
print(ms_robust[["year_month", "avg_score", "orders_count"]].head())

plt.figure(figsize=(14, 5))
plt.plot(ms_robust["year_month"], ms_robust["avg_score"], marker="o")
plt.xlabel("Mois")
plt.ylabel("Score moyen des avis")
plt.title("Évolution mensuelle du score moyen des avis")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Le score moyen progresse nettement jusqu’à mi-2017, puis connaît deux baisses distinctes.
# La première, en novembre 2017, intervient exactement au moment du pic absolu
# d’activité : volume et CA au plus haut. La surcharge logistique est donc l’explication la plus probable.
# La seconde baisse, début 2018, apparaît dans un contexte différent : le volume
# reste élevé mais le CA recule, ce qui indique un panier moyen plus faible.
# Cette combinaison (beaucoup de commandes, souvent plus petites) s’accompagne
# d’une satisfaction plus basse, possiblement liée à un mix produit plus sensible
# aux réclamations ou à une performance opérationnelle moins stable.


In [ ]:
# 3) Heatmaps année x mois : chiffre d'affaires et score moyen

# Préparation
monthly2 = (
    fva
    .groupby(["year", "month"], as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean")
    )
)


pivot_rev = monthly2.pivot(index="year", columns="month", values="revenue")
pivot_score = monthly2.pivot(index="year", columns="month", values="avg_score")

# Affichage
print("== 3) Pivot CA (année x mois) ==")
print(pivot_rev)

print("\n== Pivot score moyen (année x mois) ==")
print(pivot_score)

plt.figure(figsize=(10, 5))
sns.heatmap(pivot_rev, annot=False)
plt.title("Heatmap du CA par année et mois")
plt.xlabel("Mois")
plt.ylabel("Année")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 5))
sns.heatmap(pivot_score, annot=False)
plt.title("Heatmap du score moyen par année et mois")
plt.xlabel("Mois")
plt.ylabel("Année")
plt.tight_layout()
plt.show()


In [ ]:
# Cette double heatmap ne fait que confirmer les résultats des deux analyses précédentes :
# on retrouve la montée continue du CA jusqu’au pic de novembre 2017, puis la stabilisation
# et la fin brutale des données en 2018. Le score moyen suit la même logique que précédemment :
# il reste stable sur l’année mais chute lors des pics d’activité, en particulier autour
# du Black Friday. Ce graphe apporte donc une lecture synthétique temps × intensité qui
# renforce et valide les observations déjà établies sur la saisonnalité et la satisfaction.

In [ ]:
# 4) Distribution globale des scores d'avis

# Préparation
score_dist = (
    fva
    .groupby("score", as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum")
    )
)

# Affichage
print("== 4) Distribution des scores d'avis ==")
print(score_dist)

print("\nScore moyen global (toutes commandes) :")
print(fva["score"].mean())

plt.figure(figsize=(8, 5))
plt.bar(score_dist["score"], score_dist["orders_count"])
plt.xlabel("Score")
plt.ylabel("Nombre de commandes")
plt.title("Distribution des scores d'avis")
plt.xticks([1, 2, 3, 4, 5])
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
plt.bar(score_dist["score"], score_dist["revenue"])
plt.xlabel("Score")
plt.ylabel("Chiffre d'affaires total")
plt.title("Chiffre d'affaires par score d'avis")
plt.xticks([1, 2, 3, 4, 5])
plt.tight_layout()
plt.show()


In [ ]:
# Un graphique cette fois qualitatif centré sur la satisfaction client.
# La distribution est très asymétrique : la note 5 domine, mais la note 1 reste
# la seconde plus fréquente, ce qui reflète un comportement typique du e-commerce :
# les clients expriment surtout une forte satisfaction ou une forte insatisfaction.
# Les notes intermédiaires (2–4) sont moins représentées, souvent liées à une
# expérience correcte mais avec une légère déception (retard, produit différent, etc.).
# Les commandes bien notées concentrent l’essentiel du CA, tandis que les notes
# basses concernent surtout des achats de plus faible valeur.
# La satisfaction globale est élevée mais polarisée, révélant une sensibilité sur
# les cas où l’expérience dévie de la promesse attendue.


In [ ]:
# 5) Score moyen en fonction du délai de livraison (classes de délai)

# Préparation
tmp = fva.copy()
tmp = tmp[tmp["livree"].notna() & (tmp["livree"] > 0)]

bins = [0, 24, 72, 168, 336, np.inf]
labels = ["<=1j", "1-3j", "3-7j", "7-14j", ">14j"]

tmp["delivery_bucket"] = pd.cut(
    tmp["livree"],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

delivery_score = (
    tmp
    .groupby("delivery_bucket", as_index=False, observed=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean")
    )
)

# Affichage
print("== 5) Score moyen par classe de délai de livraison ==")
print(delivery_score.sort_values("delivery_bucket"))

plt.figure(figsize=(8, 5))
plt.bar(delivery_score["delivery_bucket"], delivery_score["avg_score"])
plt.xlabel("Délai de livraison (classes)")
plt.ylabel("Score moyen des avis")
plt.title("Score moyen en fonction du délai de livraison")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
plt.bar(delivery_score["delivery_bucket"], delivery_score["orders_count"])
plt.xlabel("Délai de livraison (classes)")
plt.ylabel("Nombre de commandes")
plt.title("Volume de commandes par classe de délai de livraison")
plt.tight_layout()
plt.show()

In [ ]:
# On voit clairement que plus la livraison est rapide, plus les clients sont satisfaits.
# Les délais courts (1–3 jours et 3–7 jours) obtiennent les meilleurs scores, ce qui correspond
# bien à ce qu’on attend dans l’e-commerce. Quand la livraison dépasse une semaine, le score
# baisse progressivement, et il chute vraiment au-delà de 14 jours. Ces deux classes longues
# regroupent en plus la majorité des commandes, ce qui explique pourquoi elles influencent
# autant la satisfaction globale observée dans les analyses précédentes. À noter que la classe
# “≤1 jour” n’est pas représentative : il y a trop peu de cas pour en tirer une conclusion fiable.
# Au final, le lien entre délai de livraison et satisfaction est très net : dès que les livraisons
# s’allongent, les avis se dégradent.

In [ ]:
# 6) CA et score moyen par mode de paiement principal (graphe combiné)

# Détermination du mode de paiement principal
pay_value_cols = [c for c in fva.columns if c.startswith("value_") and c != "value"]

def main_payment_type(row):
    sub = row[pay_value_cols]
    if sub.isna().all() or sub.max() == 0:
        return "unknown"
    return sub.idxmax().replace("value_", "")

fva["main_payment_type"] = fva[pay_value_cols].apply(main_payment_type, axis=1)

# Agrégation
pay_agg = (
    fva
    .groupby("main_payment_type", as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean"),
    )
    .sort_values("revenue", ascending=False)
)

print("== 6) Synthèse par mode de paiement principal ==")
print(pay_agg)

# Graphe unique
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(pay_agg["main_payment_type"], pay_agg["revenue"])
ax1.set_xlabel("Mode de paiement principal")
ax1.set_ylabel("Chiffre d'affaires total", color="C0")
ax1.tick_params(axis="y", labelcolor="C0")
plt.xticks(rotation=45, ha="right")

ax2 = ax1.twinx()
ax2.plot(pay_agg["main_payment_type"], pay_agg["avg_score"],
         marker="o", linestyle="-", color="C1")
ax2.set_ylabel("Score moyen des avis", color="C1")
ax2.tick_params(axis="y", labelcolor="C1")

plt.title("CA et score moyen par mode de paiement principal")
plt.tight_layout()
plt.show()


In [ ]:
# La carte de crédit concentre presque tout le chiffre d’affaires, ce qui reflète bien
# les habitudes d’achat au Brésil. Le boleto arrive loin derrière, mais avec un score
# moyen très proche : le mode de paiement n’a donc pas d’impact notable sur la
# satisfaction. Les autres moyens (voucher, débit) restent marginaux mais conservent
# des notes élevées. Le cas “unknown” n’est pas exploitable car il concerne seulement
# quatre commandes. Au global, le choix du paiement explique surtout la répartition du
# CA, tandis que les avis clients restent stables quel que soit le moyen utilisé.


In [ ]:
# 7) CA et score moyen par ÉTAT client (Top 10)

# Prépa + agrégation
state_top10 = (
    fva
    .merge(customers[["customer_id", "name_state"]], on="customer_id", how="left")
    .groupby("name_state", as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean"),
    )
    .sort_values("revenue", ascending=False)
    .head(10)
)

print("== 7) Top 10 états clients par CA ==")
print(state_top10[["name_state", "orders_count", "revenue", "avg_score"]])

# Graphe unique : CA (barres) + score moyen (courbe)
fig, ax1 = plt.subplots(figsize=(14, 6))

ax1.bar(state_top10["name_state"], state_top10["revenue"])
ax1.set_xlabel("État du client")
ax1.set_ylabel("Chiffre d'affaires", color="C0")
ax1.tick_params(axis="y", labelcolor="C0")
plt.xticks(rotation=45, ha="right")

ax2 = ax1.twinx()
ax2.plot(state_top10["name_state"], state_top10["avg_score"],
         marker="o", linestyle="-", color="C1")
ax2.set_ylabel("Score moyen des avis", color="C1")
ax2.tick_params(axis="y", labelcolor="C1")

plt.title("Top 10 états clients par CA et score moyen des avis")
plt.tight_layout()
plt.show()

In [ ]:
# Ce graphe confirme que le chiffre d’affaires se concentre surtout dans les plus
# grands bassins de population, ce qui est attendu. Le point intéressant se joue
# plutôt sur les notes : dans les zones les mieux desservies, la satisfaction reste
# élevée et assez stable, alors que dès qu’on s’éloigne des grandes villes, le
# score moyen tend à baisser légèrement. Cela suggère que la logistique devient
# plus complexe (distances, délais, fiabilité de la livraison) et que ces contraintes
# se répercutent directement dans les avis clients.


In [ ]:
# 8) CA et volume de commandes par jour de la semaine

# 0 = lundi, 6 = dimanche
fva["weekday"] = fva["purchase_timestamp"].dt.dayofweek
weekday_labels = {
    0: "Lundi",
    1: "Mardi",
    2: "Mercredi",
    3: "Jeudi",
    4: "Vendredi",
    5: "Samedi",
    6: "Dimanche"
}
fva["weekday_name"] = fva["weekday"].map(weekday_labels)

weekly = (
    fva
    .groupby("weekday", as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        revenue=("value", "sum"),
        avg_score=("score", "mean")
    )
)

weekly["weekday_name"] = weekly["weekday"].map(weekday_labels)

print("== 8) Synthèse par jour de la semaine ==")
print(weekly)

fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(weekly["weekday_name"], weekly["revenue"])
ax1.set_xlabel("Jour de la semaine")
ax1.set_ylabel("Chiffre d'affaires", color="C0")

ax2 = ax1.twinx()
ax2.plot(weekly["weekday_name"], weekly["orders_count"], marker="o", color="C1")
ax2.set_ylabel("Nombre de commandes", color="C1")

plt.title("CA et volume de commandes par jour de la semaine")
plt.tight_layout()
plt.show()


In [ ]:
# La semaine présente un cycle très régulier : le volume décroît du lundi
# au samedi, avec un minimum marqué le samedi. Pourtant, le chiffre d’affaires
# remonte nettement le dimanche, ce qui révèle un panier moyen plus élevé en fin
# de semaine. Autrement dit, les clients achètent moins le dimanche, mais ils
# réalisent des achats plus importants, ce qui compense partiellement la baisse
# de volume.

In [ ]:
# 9) Valeur moyenne de commande en fonction du score d'avis

score_value = (
    fva
    .groupby("score", as_index=False)
    .agg(
        orders_count=("order_id", "nunique"),
        avg_value=("value", "mean"),
        total_revenue=("value", "sum")
    )
)

print("== 9) Valeur moyenne et volume par score d'avis ==")
print(score_value)

plt.figure(figsize=(8, 5))
plt.bar(score_value["score"], score_value["avg_value"])
plt.xlabel("Score d'avis")
plt.ylabel("Valeur moyenne de commande")
plt.title("Valeur moyenne de commande en fonction du score d'avis")
plt.xticks([1, 2, 3, 4, 5])
plt.tight_layout()
plt.show()


In [ ]:
# Les commandes les plus chères sont aussi celles qui reçoivent le plus de notes basses.
# Le score 1 correspond au panier moyen le plus élevé, signe que les clients
# expriment davantage leur mécontentement lorsqu’il s’agit d’achats importants.
# À l’inverse, les commandes bien notées affichent un panier moyen plus faible
# et beaucoup plus stable.
# En résumé, plus la valeur d’une commande est élevée, plus le risque de déception
# et donc de mauvaise note augmente.


In [ ]:
# Conclusion générale – Analyse 1 : Performance commerciale & saisonnalité vs satisfaction

# L’ensemble des analyses met en évidence un business en forte expansion,
# porté avant tout par la croissance du volume de commandes plutôt que par le panier moyen.
# La saisonnalité joue un rôle décisif : le pic massif de novembre (Black Friday)
# stimule les ventes mais provoque également la chute la plus nette de satisfaction,
# révélant une logistique sous tension lors des périodes critiques.

# La satisfaction reste globalement élevée, mais elle dépend très fortement
# de la rapidité de livraison : dès que les délais s’allongent, les avis se dégradent.
# Géographiquement, le chiffre d’affaires se concentre dans les grands États,
# tandis que les zones plus éloignées montrent des scores légèrement plus faibles,
# signe de contraintes logistiques plus complexes.

# Enfin, les comportements d’achat indiquent que les commandes les plus chères
# sont aussi celles qui génèrent le plus de notes basses, illustrant une exigence
# accrue des clients pour les achats à forte valeur. Au final, la croissance
# commerciale tient, mais sa stabilité dépend directement de la qualité opérationnelle,
# surtout lors des pics et dans les régions difficiles à desservir.
